In [9]:
# copyrights (c) 2023 - thyung
# GPLv3

from ipywidgets import *
import time
import threading
import math
import random

SCREEN_X = 24
SCREEN_Y = 4
FRAME_RATE = 10
FIRE_SPEED_X = 2.0 / FRAME_RATE
PARTICLE_SPEED_X = 0.5 / FRAME_RATE
MIN_PARTICLE_DISTANCE = 10
MAX_MISSED = 3

class MovingObject:
    def __init__(self, x, y, speed_x, speed_y):
        self.x = x
        self.y = y
        self.speed_x = speed_x
        self.speed_y = speed_y
    def nextframe(self):
        self.x = self.x + self.speed_x
        self.y = self.y + self.speed_y
    def is_inside_screen(self):
        if 0 <= self.x and self.x < SCREEN_X and 0 <= self.y and self.y < SCREEN_Y:
            return True
        else:
            return False

class Particle(MovingObject):
    def __init__(self, x, y, speed_x, speed_y, state):
        super().__init__(x, y, speed_x, speed_y)
        self.state = state
    def __str__(self):
        return '(x,y)=({},{}), (sx,sy)=({},{}), state={}'.format(self.x, self.y, self.speed_x, self.speed_y, self.state)

class SpaceShip:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def move_up(self):
        self.y = self.y - 1
        if self.y < 0:
            self.y = self.y + SCREEN_Y
    def move_down(self):
        self.y = self.y + 1
        if SCREEN_Y <= self.y:
            self.y = self.y - SCREEN_Y
    def move_left(self):
        self.x = self.x - 1
        if self.x < 0:
            self.x = self.x + SCREEN_X
    def move_right(self):
        self.x = self.x + 1
        if SCREEN_X <= self.x:
            self.x = self.x - SCREEN_X

class Fire(MovingObject):
    def __init__(self, x, y, speed_x, speed_y, gate):
        super().__init__(x, y, speed_x, speed_y)
        self.gate = gate
    def __str__(self):
        return '(x,y)=({},{}), (sx,sy)=({},{}), gate={}'.format(self.x, self.y, self.speed_x, self.speed_y, self.gate)

def apply_gate_on_state(gate, state):
    if gate == 'X':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '+'
        elif state == '-':
            result = '-'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'
    elif gate == 'Y':
        if state == '0':
            result = '1'
        elif state == '1':
            result = '0'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == 'R':
            result = 'R'
        elif state == 'L':
            result = 'L'
    elif gate == 'Z':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = '-'
        elif state == '-':
            result = '+'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'
    elif gate == 'S':
        if state == '0':
            result = '0'
        elif state == '1':
            result = '1'
        elif state == '+':
            result = 'R'
        elif state == '-':
            result = 'L'
        elif state == 'R':
            result = '-'
        elif state == 'L':
            result = '+'
    elif gate == 'H':
        if state == '0':
            result = '+'
        elif state == '1':
            result = '-'
        elif state == '+':
            result = '0'
        elif state == '-':
            result = '1'
        elif state == 'R':
            result = 'L'
        elif state == 'L':
            result = 'R'

    return result
        
def get_particle_score(state):
    if state == '0':
        score = 1
    elif state == '1':
        score = -1
    elif state == '+' or state == '-' or state == 'R' or state == 'L':
        score = random.randrange(-1, 2, 2)  # -1 or +1
    return score

class GameControl:
    def __init__(self):
        self.ship = SpaceShip(0, 0)
        self.states = ['0', '1', '+', '-', 'R', 'L']
        self.highestscore = 0
        self.build_ui()
        self.init_values()

    def init_values(self):
        self.doublebuf = [[0 for y in range(SCREEN_Y)] for x in range(SCREEN_X)] # doublebuf[x][y]
        self.fires = []
        self.particles = []
        self.totalscore = 0
        self.txt_totalscore.value = self.totalscore
        self.missed = 0
        self.txt_missed.value = 0
        self.lbl_status.value = ''
        self.exit = False

    def build_ui(self):
        lbl_inst0 = Label(value='Summary: space ship fires "gate" to turn particles "state" to |0>')
        lbl_inst1 = Label(value='1. Move ship "D" to collect particle.')
        lbl_inst2 = Label(value='2. When ship hits particle, it measures the state and get +1 score for |0> and -1 for |1>.')
        lbl_inst3 = Label(value='3. Fire gate to turn particle state to |0> and collect it.')
        lbl_inst4 = Label(value='4. If miss 3 particles, game over.')
        box_inst = VBox([lbl_inst0, lbl_inst1, lbl_inst2, lbl_inst3, lbl_inst4])

        self.tiles = [[Button(description = ' ', layout=Layout(width='30px', height='30px'), disabled=True) for i in range(SCREEN_X)] 
                    for j in range(SCREEN_Y)]
        boxes = [Box(i) for i in self.tiles]
        board = VBox(boxes)
        box_board_inst = VBox([board, box_inst])

        btn_newgame = Button(description='new game')
        btn_up = Button(description='^', layout=Layout(width='40px'))
        btn_down = Button(description='v', layout=Layout(width='40px'))
        btn_left = Button(description='<', layout=Layout(width='40px'))
        btn_right = Button(description='>', layout=Layout(width='40px'))
        panel_updown = VBox([btn_up, btn_down])
        panel_dir = Box([btn_left, panel_updown, btn_right], layout=Layout(align_items='center'))
        btn_x = Button(description='X')
        btn_y = Button(description='Y')
        btn_z = Button(description='Z')
        btn_s = Button(description='S')
        btn_h = Button(description='H')
        btn_exit = Button(description='Exit')
        btn_particle = Button(description='Particle')
        self.txt_totalscore = BoundedIntText(value=0, description='score:', layout=Layout(width='140px'), disabled=True)
        self.txt_highestscore = BoundedIntText(value=0, description='highest:', layout=Layout(width='140px'), disabled=True)
        self.txt_missed = BoundedIntText(value=0, description='missed:', layout=Layout(width='140px'), disabled=True)
        self.lbl_status = Label(value='')
        panel_ctrl = VBox([btn_newgame, 
                            panel_dir, 
                            btn_x, btn_y, btn_z, btn_s, btn_h, 
                            # btn_exit, 
                            # btn_particle, 
                            self.txt_totalscore, self.txt_highestscore, 
                            self.txt_missed, self.lbl_status],
                            layout=Layout(align_items='center'))
        screen = Box([panel_ctrl, box_board_inst], layout=Layout(justify_content='space-between', align_items='center'))
        display(screen)

        btn_newgame.on_click(self.on_click_newgame)
        btn_up.on_click(lambda b: self.ship.move_up())
        btn_down.on_click(lambda b: self.ship.move_down())
        btn_left.on_click(lambda b: self.ship.move_left())
        btn_right.on_click(lambda b: self.ship.move_right())

        btn_x.on_click(self.on_click_x)
        btn_y.on_click(self.on_click_y)
        btn_z.on_click(self.on_click_z)
        btn_s.on_click(self.on_click_s)
        btn_h.on_click(self.on_click_h)

        btn_exit.on_click(self.on_click_exit)
        btn_particle.on_click(self.on_click_particle)

    def on_click_newgame(self, b):
        self.init_values()
        threading.Thread(target=self.loop).start()

    def on_click_x(self, b):
        fire = Fire(self.ship.x + 1, self.ship.y, FIRE_SPEED_X, 0, 'X')
        self.fires.append(fire)

    def on_click_y(self, b):
        fire = Fire(self.ship.x + 1, self.ship.y, FIRE_SPEED_X, 0, 'Y')
        self.fires.append(fire)

    def on_click_z(self, b):
        fire = Fire(self.ship.x + 1, self.ship.y, FIRE_SPEED_X, 0, 'Z')
        self.fires.append(fire)

    def on_click_s(self, b):
        fire = Fire(self.ship.x + 1, self.ship.y, FIRE_SPEED_X, 0, 'S')
        self.fires.append(fire)

    def on_click_h(self, b):
        fire = Fire(self.ship.x + 1, self.ship.y, FIRE_SPEED_X, 0, 'H')
        self.fires.append(fire)

    def on_click_exit(self, b):
        self.exit = True

    def on_click_particle(self, b):
        particle = Particle(SCREEN_X - 1, self.ship.y, -PARTICLE_SPEED_X, 0, '0' )
        self.particles.append(particle)

    def nextframe_fires(self):
        for fire in self.fires:
            fire.nextframe()
        self.fires = [fire for fire in self.fires if fire.is_inside_screen()]
    def nextframe_particles(self):
        for particle in self.particles:
            particle.nextframe()
        particles_missed = [p for p in self.particles if not p.is_inside_screen()]
        missed = len(particles_missed)
        if 0 < missed:
            self.missed = self.missed + missed
            self.txt_missed.value = self.missed
        if MAX_MISSED <= self.missed:
            self.lbl_status.value = 'GAME OVER'
            self.exit = True
        self.particles = [particle for particle in self.particles if particle.is_inside_screen()]

    def check_fire_collision(self):
        for fire in self.fires:
            fire_x = math.floor(fire.x)
            fire_y = math.floor(fire.y)
            for particle in self.particles:
                if math.floor(particle.x) == fire_x and math.floor(particle.y) == fire_y:
                    result = apply_gate_on_state(fire.gate, particle.state)
                    particle.state = result
                    self.fires.remove(fire)
                    break

    def check_ship_collision(self):
        ship_x = math.floor(self.ship.x)
        ship_y = math.floor(self.ship.y)
        for particle in self.particles:
            if math.floor(particle.x) == ship_x and math.floor(particle.y) == ship_y:
                score = get_particle_score(particle.state)
                self.totalscore = self.totalscore + score
                if self.totalscore < 0:
                    self.totalscore = 0
                if self.highestscore < self.totalscore:
                    self.highestscore = self.totalscore
                    self.txt_highestscore.value = self.highestscore
                self.txt_totalscore.value = self.totalscore
                self.particles.remove(particle)
                break

    def gen_particle(self):
        r = random.randrange(0, 100)
        if r <= self.totalscore or self.particles == []:
            x = SCREEN_X - 1
            y = random.randrange(0, SCREEN_Y)
            if [p for p in self.particles if SCREEN_X - MIN_PARTICLE_DISTANCE <= p.x and p.y == y] == []:
                particle = Particle(x, y, -PARTICLE_SPEED_X, 0, random.choice(self.states))
                self.particles.append(particle)

    def draw_screen(self, x, y, v):
        self.tiles[math.floor(y)][math.floor(x)].description = v

    def init_screen(self):
        for y in range(SCREEN_Y):
            for x in range(SCREEN_X):
                self.draw_screen(x, y, ' ')

    def draw_frame(self):
        for y in range(SCREEN_Y):
            for x in range(SCREEN_X):
                self.doublebuf[x][y] = ' '
        for fire in self.fires:
            self.doublebuf[math.floor(fire.x)][math.floor(fire.y)] = fire.gate
        for particle in self.particles:
            self.doublebuf[math.floor(particle.x)][math.floor(particle.y)] = particle.state
        self.doublebuf[self.ship.x][self.ship.y] = 'D'

        for y in range(SCREEN_Y):
            for x in range(SCREEN_X):
                self.draw_screen(x, y, self.doublebuf[x][y])

    def loop(self):
        while not self.exit:
            # self.init_screen()
            self.nextframe_fires()
            self.check_fire_collision()
            self.nextframe_particles()
            self.check_fire_collision()
            self.gen_particle()
            self.check_ship_collision()
            self.draw_frame()
            time.sleep(1.0 / FRAME_RATE)

game_control = GameControl()

threading.Thread(target=game_control.loop).start()


Box(children=(VBox(children=(Button(description='new game', style=ButtonStyle()), Box(children=(Button(descrip…